<a href="https://colab.research.google.com/github/Skaipe12/Data_science/blob/main/Backend_Amazon_WebApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn

In [ ]:
!pip install mysql-connector-python

In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import mysql.connector
from pydantic import BaseModel
from typing import List
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

Endpoint1

In [ ]:
class ItemsList(BaseModel):
    items: List[int]

app = FastAPI()
app.add_middleware(CORSMiddleware,allow_origins=["*"],allow_credentials=True,allow_methods=["*"],allow_headers=["*"])
user_item_matrix = pd.read_csv("/content/user_item_matrix.csv")

def recomended_prod(car_items):
  cart_vector = user_item_matrix.iloc[:, car_items].sum(axis=1)
  user_item_matrix_t=user_item_matrix.transpose()
  similitary_scores=cosine_similarity(cart_vector.values.reshape(1,-1),user_item_matrix_t)
  similiar_users=similitary_scores.argsort()[0][::-1][:3]

  similar_users_matrix = user_item_matrix.iloc[similiar_users, :]
  similar_users_vector = similar_users_matrix.sum(axis=0)
  already_purchased_items=set(car_items)

  recomended_items=set(similar_users_vector.index)-already_purchased_items
  recomended_items=list(recomended_items)[:3]
  return recomended_items


@app.get("/")
async def root():
    mydb=mysql.connector.connect(host="aa18opju4pam1ik.c2xacwacu1dj.us-west-2.rds.amazonaws.com", user="",password="",database="")
    mycursor=mydb.cursor(buffered=True,dictionary=True)

    mycursor.execute("SELECT * FROM ebdb.flashpedidos_product_info LIMIT 10")
    list_product=mycursor.fetchall()
    send_list=[]
    for i in list_product:
        dict_1={}
        dict_1["id"]=i["id"]
        dict_1["product_name"]=i["product_name"]
        dict_1["product_price"]=i["product_price"]
        id=i["id"]
        mycursor.execute("SELECT * FROM ebdb.flashpedidos_image_product_info WHERE id_product_id=%s",(id,))
        image_product=mycursor.fetchall()
        dict_1["product_image"]=image_product[0]["product_picture"]
        send_list.append(dict_1)

    return{"data":send_list}

Endpoint 2 items recomendados

In [ ]:
@app.post('/recomended_products')
async def recomended_products(items: ItemsList):
  print(items)
  mydb=mysql.connector.connect(host="aa18opju4pam1ik.c2xacwacu1dj.us-west-2.rds.amazonaws.com", user="flashadmin",password="est23loz55",database="ebdb")
  mycursor=mydb.cursor(buffered=True,dictionary=True)
  item_array=items.items
  recomended_products=recomended_prod(item_array)
  send_list=[]
  for i in recomended_products:
    try:
        mycursor.execute("SELECT * FROM ebdb.flashpedidos_product_info WHERE id = %s", (int(i),))
        product=mycursor.fetchall()
        print(product[0])
        dict_1={}
        dict_1["id"]=product[0]["id"]
        dict_1["product_name"]=product[0]["product_name"]
        dict_1["product_price"]=product[0]["product_price"]
        id=product[0]["id"]
        mycursor.execute("SELECT * FROM ebdb.flashpedidos_image_product_info WHERE id_product_id = %s", (id,))
        image_product=mycursor.fetchall()
        dict_1["image_product"]=image_product
        send_list.append(dict_1)
    except:
      print("Producto no existe")



  return {"result":send_list}

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrook_tunnel= ngrok.connect(8000)
print("servidor: ",ngrook_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app,port=8000)